#### **0. Import Libraries and define Options**

***Import Libraries***

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import scipy.stats

***Define some options***

In [ ]:
# Set initial Training samples
IDX_INIT_MODEL = 500

# Optimal PC Selection Method
Opt_PC_select_method = 'eigengap'

# Significance level for Q & T2 score
# alpha = 0.95
alpha = 0.99

#### **1. Load Dataset**

In [ ]:
import pickle

# load
fn_load = 'J_Dataset_1101_0630_outlier_3sig'

with open(fn_load + '.pickle', 'rb') as f:
    data = pickle.load(f)

fn = data['fn']
damage_ind = data['damage_ind']
df1, df3 = data['data'][0], data['data'][1]

#### **2. Set Dataset (Train & Test)**

***Define Dataset***

In [ ]:
if 1:
    col_interest = ['Time', 'CG_1', 'CG_2', 'TT_1', 'TT_2']
    df = df1[col_interest]
    Label = df1.Label.values

else:
    col_interest = ['Time', 'CG_3', 'CG_4', 'TT_3', 'TT_4']
    df = df3[col_interest]
    Label = df3.Label.values

X_all = df.values[:, 1:]

***Find damage index as # index of sample***

In [ ]:
damage_ind = []
for ind_label in np.unique(Label):
    if ind_label != 0:
        ind_damage = np.where(Label == ind_label)[0][0]
        damage_ind.append(ind_damage)
damage_ind

***Plot Ground Truth (label)***

In [ ]:
# Plot scatter plot (Time index vs. Label)
color_type_str = ['blue', 'orange', 'red']

plt.figure(figsize = (10, 3), dpi = 200)
for label_ind in np.unique(Label):
    indice_ = np.where(Label == label_ind)
    plt.plot(df.Time.iloc[indice_], Label[indice_], marker = '.', color = color_type_str[label_ind])
plt.xlabel('Time')
plt.ylabel('Label')
plt.gca().set_yticks([0, label_ind])
plt.grid(linestyle = ':')
if 'CG_1' in df.columns:
    struct_type = 'Caisson #1'
else:
    struct_type = 'Caisson #3'

plt.title(struct_type)
plt.show()

***Define Training and Test Dataset***

In [ ]:
# Set last index for normal state
IDX_NORMAL = damage_ind[0] - 1

if IDX_INIT_MODEL > IDX_NORMAL:
    IDX_INIT_MODEL = IDX_NORMAL
    print(f'Index for Normal state should be less than index of damage {damage_ind[0]}')

# For Allocation of memory
SIZE_ALL = X_all.shape[0]

***Plot Rawdata in line plot***

In [ ]:
color_type_str = ['blue', 'orange', 'red']

for col_ind in range(X_all.shape[1]):
    plt.figure(figsize = (10, 3), dpi = 200)
    for label_ind in np.unique(Label):
        row_ind = np.where(Label == label_ind)
        plt.plot(df.Time.iloc[row_ind], X_all[row_ind, col_ind].reshape(-1, 1),
                marker = '.', color = color_type_str[label_ind])
    plt.xlabel('Time')
    plt.ylabel(col_interest[col_ind + 1])
    plt.grid(linestyle = ':')
    plt.title(struct_type)
    plt.show()


#### **3. Set Initial Dataset & Scaling (Standardization)**

***Set Initial Traininig Dataset***

In [ ]:
from sklearn.preprocessing import StandardScaler

# Define standardizer (scaling)
scaler = StandardScaler()
Xtrain = scaler.fit_transform(X_all[0:IDX_INIT_MODEL,:])

plt.figure(figsize = (10, 5), dpi =150)
plt.plot(Xtrain, marker = '.', label = col_interest[1:])
plt.grid(linestyle = ':')
plt.xlabel('# Index')
plt.title(f'Stadardized Training data {struct_type}')
plt.legend()
plt.show()

***Set Initial Test Dataset***

In [ ]:
Xtest = X_all[IDX_INIT_MODEL:,:]

#### **4. Construct Baseline model using initial trainig dataset**

***Helper function for ftting PCA***

In [ ]:
def perform_PCA_given_data(X):
    '''
        - Input
            X : Normalzied training samples [N(# samples)-by-f(# features)]

        - Output
            pca: fitted PCA model
            explVar: exaplined variance for PCs
            V: principal compoent vectors
            n_comp: # retained PCs
    '''
    from sklearn.decomposition import PCA
    pca = PCA().fit(X)

    # Explained variance of each features for PCA
    expVars = pca.explained_variance_
    
    # Retained PCs
    V = pca.components_
    
    # Singular values
    S = pca.singular_values_

    return pca, expVars, V, S

***Run PCA for initial training dataset***
- pca: PCA class instance
- expVARs: Explained variances ($=\frac{S_i}{\sum S_i}$)
- V: Principal Component Vectors
- S: Singular Values

In [ ]:
pca, expVars, V, S = perform_PCA_given_data(Xtrain)

***Select optimal # of PC: n_comp***

In [ ]:
if Opt_PC_select_method == 'eigengap':
    # Select optimal # of PCs: eigengap technique
    # % ref.1) The rotation of eigenvectors by a perturbation (1970)
    # % ref.2) Adaptive data-derived anomaly detection in the activated... (2016)
    n_comp = np.argmax(np.abs(np.diff(expVars)))

print(f'# optimal PC: {n_comp + 1}')
retaind_PCs = np.arange(n_comp + 1)

#### **Plot exlained PCs**

In [ ]:
fig, ax = plt.subplots(2, 1, figsize = (10, 7))
ax[0].bar(np.arange(0, S.shape[0]), S)
ax[0].set_ylabel('Eigenvalue (SV)')
# ax[0].bar(np.arange(0, expVars.shape[0]), expVars / sum(expVars))
# ax[0].set_ylabel('Explained Variance')
ax[0].set_xlabel('# PCs')
ax[0].grid(linestyle = ':')


ax[1].plot(np.arange(0, S.shape[0]-1), np.abs(np.diff(S)))
ax[1].plot(n_comp, np.abs(np.diff(S))[n_comp], 'bo', label = 'Optimal # PC')
ax[1].set_ylabel('Difference of Eigenvalue (Eigengap)')
# ax[1].plot(np.arange(0, S.shape[0]-1), np.abs(np.diff(expVars)))
# ax[1].plot(n_comp, np.abs(np.diff(expVars))[n_comp], 'bo', label = 'Optimal # PC')
# ax[1].set_ylabel('Difference of Explained Variance')
ax[1].set_xlabel('# PCs')
ax[1].grid(linestyle = ':')
ax[1].legend()
plt.show()

***Compute Monitroing metrics and their thresholds***

In [ ]:
def Project_X_into_PCs(X, V, retaind_PCs):
    return np.matmul(X, V[retaind_PCs].T)

***Prject dataset into retained PCs***

In [ ]:
X_proj = Project_X_into_PCs(Xtrain, V, retaind_PCs)
X = Xtrain

$$
    Q(X, P_{1:r}) = X(I - P_{1:r}P_{1:r}^T)X^T = \| X - \hat{X} \|^2
$$

where $\hat{X} = P_{1:r} X$
- $X$: Standardized X
- $P$: Principal component (PC) vectors
- $r$: # of retained PCs

*1.1) Compute Q-Statistics*

In [ ]:
def compute_Q_statistics(X, X_proj, V, retaind_PCs):
    return np.sqrt(np.sum(
    (X - np.matmul(X_proj, V[retaind_PCs]))**2
    , axis = 1))

In [ ]:
Q = compute_Q_statistics(X, X_proj, V, retaind_PCs)

*1.2) Compute threshold of Q-Statistics*

$$
    Q_{alpha} = \frac{\theta_2}{2 \theta_1}\chi_{\alpha}^2(h)
$$

where,
- $\theta_1$: sample mean
- $\theta_2$: sample variance
- $\chi_{\alpha}^2(h)$: chi-squared distribution with $h$ degree of freedom and the significance level $\alpha$
- $h = \frac{2\theta_1^2}{\theta_2}$

> Note it is based on normality assumption on Q-statistics

In [ ]:
def compute_Threshold_Q(Q, alpha):
    from scipy.stats.distributions import chi2
    
    theta1 = np.mean(Q)
    theta2 = np.var(Q)
    h = 2 * (theta1 ** 2) / theta2
    chi_h = chi2.ppf(alpha, df=h)
    Qlimit = theta2/(2*theta1) * chi_h
    return Qlimit

In [ ]:
Qlimit = compute_Threshold_Q(Q, alpha) # scalar
Qlimit = np.ones_like(Q) * Qlimit # Make it `scalar of Qlimit` to `same vector size of training data (Q1)`

In [ ]:
plt.figure(figsize = (10, 3), dpi = 200)
plt.plot(Q, '.')
plt.plot(Qlimit, color = 'r', linestyle = '-')
plt.grid(linestyle = ':')
plt.xlabel('# Sample Index')
plt.ylabel('Q-statistics')
plt.title(f'Q-statistics $\\alpha = {alpha}$')
plt.show()

**2) Hotelling’s T2 Statistic**

Now to calculate the T2 statistic, just transform each example.

We calculate the SVD decomposition of the covariance matrix, and with that we can use the equation below to calculate the z_score to each example in our dataset

$$
    z = \Lambda_{1:r}^{-1/2}P_{1:r}^TX, \\
    \text{ } \\
    T^2 =z^Tz
$$

where,
- X: Standardized X
- $\Lambda$:
- $P$: Principal component (PC) vectors
- $r$: # of retained PCs

*2.1) Compute T2-Statistics*

In [ ]:
def compute_T2_statistics(X_proj, S, retaind_PCs):
    S_inverse = np.mat(np.diag(S[retaind_PCs])).I

    T2 = np.empty((X_proj.shape[0], ), dtype=np.float64)
    for i in range(X_proj.shape[0]):
        value = np.matmul(np.matmul(X_proj[i], S_inverse), X_proj[i])
        T2[i] = float(value)

    return T2

In [ ]:
T2 = compute_T2_statistics(X_proj, S, retaind_PCs)

*2.2) Compute threshold of T2-Statistics*

$$
    T_{\alpha}^{2} = \frac{m(n-1)(n+1)}{n(n-m)} F_{\alpha}(m, n - m)
$$

where,
- $n$: # of samples
- $m$: # of retained PCs
- $F_{\alpha}(m, n-m)$: F-distribution with $r$ and $(n-m)$ degrees of freedom with significance level $\alpha$

In [ ]:
def compute_Threshold_T2(T2, retaind_PCs, alpha):
    Ntrain, dim = T2.shape[0], retaind_PCs.shape[0]
    t2limit = ((dim*(Ntrain-1))/ (Ntrain - dim)) * \
        scipy.stats.f.ppf(q=alpha, dfn=dim, dfd=Ntrain - dim)
    
    return t2limit

In [ ]:
t2limit = compute_Threshold_T2(T2, retaind_PCs, alpha)
t2limit = np.ones_like(T2) * t2limit # Make it `scalar of t2limit` to `same vector size of training data (T2)`

In [ ]:
plt.figure(figsize = (10, 3), dpi = 200)
plt.plot(T2, '.')
plt.plot(t2limit, color = 'r', linestyle = '-')
plt.grid(linestyle = ':')
plt.xlabel('# Sample Index')
plt.ylabel('T2-statistics')
plt.title(f'T2-statistics $\\alpha = {alpha}$')
plt.show()

#### **Evaluation Step (Monitoring Phase)**

*Plot result*

- Comformal prediction => stoppting criteria => based on significance level (alpha)

In [ ]:
# scaler
# Xtrain
# Xtest
# X_scaled
# pca, expVars, V, S

# Define standardizer (scaling)
# scaler = StandardScaler()
# Xtrain = scaler.fit_transform(X_all[0:IDX_INIT_MODEL,:])
damage_ind

In [ ]:
# Q = compute_Q_statistics(X, X_proj, V, retaind_PCs) # output: Vector
# Qlimit = compute_Threshold_Q(Q, alpha) # output: Scalar

# T2 = compute_T2_statistics(X_proj, S, retaind_PCs) # output: Vector
# t2limit = compute_Threshold_T2(T2, retaind_PCs, alpha) # output: Scalar